In [1]:
import tensorflow as tf

## 하이퍼 파라미터 설정

In [2]:
# Specify hyperparameters
EPOCHS = 5

## 네트워크 구조 정의

In [26]:
# Define network architecture
class Mymodel(tf.keras.Model) :
    def __init__(self) :
        super(Mymodel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x) :
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [35]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy) :
    with tf.GradientTape() as tape :
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 루프 정의

In [36]:
# Implement alogorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy) :
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 가져오고, 정리

In [37]:
# Import and organize dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 네트워크 생성

In [38]:
# Create model
model = Mymodel()

## 손실함수, 최적화 알고리즘 정의

In [39]:
# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 알고리즘 평가지표 설정

In [40]:
# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 알고리즘 학습, 테스트 루프

In [42]:
# Do training loop and test
for epoch in range(EPOCHS) :
    for images, labels in train_ds :
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds :
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss : {}. Accuracy : {}, Test Loss : {}, Test Accuracy : {}'
    print(template.format(epoch+1, \
                                          train_loss.result(), \
                                          train_accuracy.result()*100, \
                                          test_loss.result(), \
                                          test_accuracy.result()*100))

Epoch 1, Loss : 0.29748085141181946. Accuracy : 92.01333618164062, Test Loss : 0.19183921813964844, Test Accuracy : 94.41000366210938
Epoch 2, Loss : 0.24617715179920197. Accuracy : 93.30110931396484, Test Loss : 0.1712360680103302, Test Accuracy : 95.00333404541016
Epoch 3, Loss : 0.21243755519390106. Accuracy : 94.17625427246094, Test Loss : 0.15616659820079803, Test Accuracy : 95.40750122070312
Epoch 4, Loss : 0.1878129094839096. Accuracy : 94.83666229248047, Test Loss : 0.14479175209999084, Test Accuracy : 95.72200012207031
Epoch 5, Loss : 0.168656125664711. Accuracy : 95.3647232055664, Test Loss : 0.13501843810081482, Test Accuracy : 95.98500061035156
